In [1]:
!pip install networkx

In [2]:
!pip install torch-geometric

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.1/63.1 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 16.7 MB/s eta 0:00:00


In [3]:
!pip install rdkit

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.2/35.2 MB 19.1 MB/s eta 0:00:00


### 1. Create a protein-protein interaction network

In [ ]:
!wget https://ftp.uniprot.org/pub/databases/uniprot/current_release/knowledgebase/complete/uniprot_sprot.dat.gz

--2025-05-13 13:10:02--  https://ftp.uniprot.org/pub/databases/uniprot/current_release/knowledgebase/complete/uniprot_sprot.dat.gz
Resolving ftp.uniprot.org (ftp.uniprot.org)... 128.175.240.195
Connecting to ftp.uniprot.org (ftp.uniprot.org)|128.175.240.195|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 682638053 (651M) [application/x-gzip]
Saving to: ‘uniprot_sprot.dat.gz’

uniprot_sprot.dat.g 100%[===================>] 651.01M  62.7MB/s    in 11s     

2025-05-13 13:10:13 (60.0 MB/s) - ‘uniprot_sprot.dat.gz’ saved [682638053/682638053]



In [ ]:
!wget https://downloads.thebiogrid.org/Download/BioGRID/Latest-Release/BIOGRID-ORGANISM-LATEST.mitab.zip

--2025-05-13 13:10:13--  https://downloads.thebiogrid.org/Download/BioGRID/Latest-Release/BIOGRID-ORGANISM-LATEST.mitab.zip
Resolving downloads.thebiogrid.org (downloads.thebiogrid.org)... 173.255.198.187
Connecting to downloads.thebiogrid.org (downloads.thebiogrid.org)|173.255.198.187|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/download]
Saving to: ‘BIOGRID-ORGANISM-LATEST.mitab.zip’

BIOGRID-ORGANISM-LA     [             <=>    ] 171.47M  67.6MB/s    in 2.5s    

2025-05-13 13:10:16 (67.6 MB/s) - ‘BIOGRID-ORGANISM-LATEST.mitab.zip’ saved [179801435]



In [ ]:
!unzip -p BIOGRID-ORGANISM-LATEST.mitab.zip BIOGRID-ORGANISM-Homo_sapiens-4.4.245.mitab.txt > BIOGRID-ORGANISM-Homo_sapiens-4.4.245.mitab.txt

In [ ]:
!wget https://reactome.org/download/current/interactors/reactome.homo_sapiens.interactions.psi-mitab.txt

--2025-05-13 13:12:56--  https://reactome.org/download/current/interactors/reactome.homo_sapiens.interactions.psi-mitab.txt
Resolving reactome.org (reactome.org)... 100.25.71.177
Connecting to reactome.org (reactome.org)|100.25.71.177|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 184341247 (176M) [text/plain]
Saving to: ‘reactome.homo_sapiens.interactions.psi-mitab.txt’

reactome.homo_sapie 100%[===================>] 175.80M  55.8MB/s    in 3.3s    

2025-05-13 13:12:59 (53.3 MB/s) - ‘reactome.homo_sapiens.interactions.psi-mitab.txt’ saved [184341247/184341247]



In [ ]:
!git clone https://github.com/jjjk123/GBA-centrality.git

Cloning into 'GBA-centrality'...
remote: Enumerating objects: 527, done.
remote: Counting objects: 100% (153/153), done.
remote: Compressing objects: 100% (101/101), done.
remote: Total 527 (delta 67), reused 74 (delta 52), pack-reused 374 (from 1)
Receiving objects: 100% (527/527), 5.60 MiB | 11.50 MiB/s, done.
Resolving deltas: 100% (315/315), done.


In [ ]:
!gunzip -c uniprot_sprot.dat.gz | python GBA-centrality/Interactome/uniprot_parser.py > uniprot_parsed.tsv

2025-05-13 13:13:12 INFO uniprot_parser.py: Starting to run
2025-05-13 13:20:31 INFO uniprot_parser.py: All done, completed successfully!


In [ ]:
!python GBA-centrality/Interactome/interaction_parser.py --interaction_file BIOGRID-ORGANISM-Homo_sapiens*.mitab.txt --uniprot_file uniprot_parsed.tsv > interactions_Biogrid.tsv

2025-05-13 13:20:31 INFO interaction_parser.py: Parsing Uniprot file
2025-05-13 13:20:31 INFO interaction_parser.py: Parsing interaction file
2025-05-13 13:20:31 INFO interaction_parser.py: Done!


In [ ]:
!python GBA-centrality/Interactome/interaction_parser.py --interaction_file reactome.homo_sapiens.interactions.psi-mitab.txt --uniprot_file uniprot_parsed.tsv > interactions_Reactome.tsv

2025-05-13 13:20:31 INFO interaction_parser.py: Parsing Uniprot file
2025-05-13 13:20:31 INFO interaction_parser.py: Parsing interaction file
2025-05-13 13:20:33 INFO interaction_parser.py: Done!


In [ ]:
!python GBA-centrality/Interactome/build_interactome.py \
  --interactions_parsed_files interactions_Biogrid.tsv interactions_Reactome.tsv \
  --uniprot_file uniprot_parsed.tsv > interactome_human.sif

2025-05-13 13:20:34 INFO build_interactome.py: Parsing Uniprot file
2025-05-13 13:20:34 INFO build_interactome.py: Parsing interaction files
2025-05-13 13:20:35 INFO build_interactome.py: Removing protein hubs
2025-05-13 13:20:35 INFO build_interactome.py: Printing interactome
2025-05-13 13:20:35 INFO build_interactome.py: Done!


### 2. Test the first GNN: XGDAG

In [4]:
!git clone https://github.com/GiDeCarlo/XGDAG.git
%cd /content/XGDAG

Cloning into 'XGDAG'...
remote: Enumerating objects: 2849, done.
remote: Counting objects: 100% (79/79), done.
remote: Compressing objects: 100% (63/63), done.
remote: Total 2849 (delta 30), reused 49 (delta 15), pack-reused 2770 (from 1)
Receiving objects: 100% (2849/2849), 210.71 MiB | 19.37 MiB/s, done.
Resolving deltas: 100% (1167/1167), done.
Updating files: 100% (1360/1360), done.


In [5]:
### How to fix XGDAG
!touch XGDAG/SubgraphX/__init__.py
!sed -i 's/from SubgraphXshapley/from .SubgraphXshapley/' XGDAG/SubgraphX/SubgraphX.py

In [8]:
!python TrainerScript.py -h

[+] Training C0009402
[+] Reading graph...ok
/content/XGDAG/CreateDatasetv2_binary_diamond.py:81: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  nedbit_scores["label"].replace(to_replace = 1, value = "P", inplace = True)
[+] Creating dataset...ok
[i] Elapsed time: 13.926
  0% 0/40001 [00:00<?, ?it/s][Epoch: 0000] train loss: 0.5769, train acc: 0.9214, val loss: 0.0927, val acc: 0.9965  (best train acc: 0.9214, best val acc: 0.9965, best train loss: 0.5769  @ epoch 0 )
  0% 20/40001 [00:23<11:32:26,  1.04s/it][Epoch: 0020] train

In [6]:
!python ComputeRankingScript.py ataxia gnnexplainer 1

/content/XGDAG
/content/XGDAG
[i] Computing the ranking for ['C0006142'] ( 1 ) disease(s).
[i] Starting C0006142 with method gnnexplainer
[+] A raking for disease C0006142 has already been computed with gnnexplainer. Do you want to overwrite the old ranking? (y|n) y
[+] Reading graph...ok
[+] Creating dataset...Traceback (most recent call last):
  File "/content/XGDAG/ComputeRankingScript.py", line 145, in <module>
    ranking(disease_Id, METHOD, num_cpus, filename, modality)
  File "/content/XGDAG/ComputeRankingScript.py", line 62, in ranking
    dataset, G = get_dataset_from_graph(graph_path, disease_Id, quartile=False, from_diamond=True)
                 ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/content/XGDAG/CreateDatasetv2.py", line 72, in get_dataset_from_graph
    seed_genes = pd.read_csv(path_to_seed_genes, header=None, sep=' ')
                 ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.